In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader


## 1. 문서 내용 읽기고 분할 #################################################
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 백터 데이터베이스에 저장 ####################################
## 2.1. 환경변수 읽어오기
load_dotenv()

## 2.2. 임베딩 모델 지정  
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

## 로컬에 저장된 임베딩 데이터 가져오기
database = Chroma(
    collection_name='chroma-law',
    persist_directory='./chroma',
    embedding_function=embedding,
)

## 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ######################
# 3.1. 사용자 질문
# query = '전세사기피해자 금융지원에 대해 설명해주세요.'
query = '전세사기피해자로 인정받기 위한 조건은?'

## 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=3)

## 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###################
## 4.1. 프롬프트 작성  
prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question : {query}
'''

## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)

## 4.3. LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄  
ai_message = llm.invoke(formatted_prompt)
ai_message